### Vectorize annotations
This notebook creates usage embeddings for all usages in the annotation data.



### Usage
Set the variable `annotation_csv` to the according results file of the according human annotation.

### Disclaimer
This notebook was changed to create different types of usage embeddings. The current version replaces the target word with the headword in the word usage before creating the embedding.

In [2]:
import json 
import pandas as pd
import sys

sys.path.insert(1, '../xl-lexeme/WordTransformer')

from InputExample import InputExample
from WordTransformer import WordTransformer

model = WordTransformer('pierluigic/xl-lexeme') # load model

/home/jonathan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## phase 1
annotation_csv = "../data/annotation_results/phase1/english_assigned.csv"
# annotation_csv = "../data/annotation_results/phase1/swedish_assigned.csv"

## phase 2
#annotation_csv = "../data/annotation_results/phase2/english_assigned.csv"
# annotation_csv = "../data/annotation_results/phase2/round2_swedish_assigned.csv"

output_file = f"../data/outputs/annotation_phase_1/usage_embeddings/{annotation_csv.split('/')[-1].split('.')[0]}_usage_embeddings[SUB].json"

In [4]:
annotations = pd.read_csv(annotation_csv, sep="\t") # read csv
annotations = annotations.to_dict(orient="records") # convert to dict

results = []


for annotation in annotations:
    pos1, pos2 = list(map(int, annotation['indices_target_token'].split(':'))) # get positions of target word
    context = ''.join((annotation['context'][:pos1], annotation['lemma'], annotation['context'][pos2:])) # get substituted context
    pos1, pos2 = (pos1, pos1 + len(annotation['lemma'])) # get new positions of headword in context

    embedding = model.encode(InputExample(texts=context, positions=[pos1, pos2])) # generate embedding
    results.append(
        {
            "lemma": annotation['lemma'],
            "dataID": annotation['usageID'],
            "gold_senses": annotation['sense'],
            "embedding": embedding.tolist()
        })

In [7]:
with open(output_file, "w", encoding='utf-8') as f:
    json.dump(results, f, indent=4, ensure_ascii=False)